In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as func
from scipy.io import loadmat
import numpy as np
from sklearn.feature_extraction import image
import pandas as pd
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
import string
import gensim

In [2]:
import re
import pickle
import json
import xml.etree.ElementTree as ET
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import os
from nltk.tokenize import word_tokenize, sent_tokenize
import numpy as np
import math
from difflib import SequenceMatcher
import pandas as pd

In [3]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
year_regex = re.compile(r'((19[0-9]{2})|(20[0-9]{2}))[a-z]?')
conversion_dict = {}
stop_words = [',', '.', '(', ')', ':', '-', "+", ";", "a", "about", "al", "al.", "all", 
	"already", "also", "although", "am", "an", "and", "another", "any", "anyhow", "are", 
	"aren", "aren't", "around", "as", "at", "back", "be", "because", "been", 
	"being", "beyond", "but", "by", "can", "cannot", "cant", "co", "con", "could", "couldn", 
	"couldnt", "d", "de", "did", "didn", "didn't", "do", "does", "doesn", "doesn't", 
	"doing", "don", "don't", "done", "due", "each", "either", "else", "elsewhere", "et", 
	"etc", "even", "ever", "except", "for", "found", "from", "further", "had", "hadn", 
	"hadn't", "has", "hasn", "hasn't", "hasnt", "have", "haven", "haven't", "having", 
	"he", "hence", "her", "here", "hereafter", "hereby", "hers", 
	"herself", "him", "himself", "his", "how", "however", "i", "ie", "if", "in", "inc", 
	"indeed", "interest", "into", "is", "isn", "isn't", "it", "it's", "its", "itself", 
	"just", "ltd", "ll", "m", "may", "me", "meanwhile", "might", "mightn", 
	"mightn't", "mine", "moreover", "most", "mostly", "move", "much", "must", "mustn", 
	"mustn't", "my", "myself", "name", "namely", "need", "needn", "needn't", "neither", 
	"nevertheless", "no", "nobody", "noone", "nor", "not", "now", "nowhere", "o", "of", 
	"off", "often", "on", "only", "onto", "or", "other", "others", "otherwise", "our", "own", 
	"per", "perhaps", "put", "rather", "re", "s", "same", "see", "seem", "seemed", 
	"seeming", "seems", "serious", "she", "should", "shouldn", "shouldn't", "since", 
	"sincere", "so", "some", "somehow", "someone", "something", "somewhere", "still", 
	"such", "t", "take", "than", "that", "that'll", "the", "their", "theirs", 
	"them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", 
	"therein", "thereupon", "these", "they", "this", "those", "though", "throughout", 
	"thru", "thus", "to", "together", "too", "toward", "towards", "un", "until", "upon", 
	"us", "ve", "very", "via", "was", "wasn", "wasn't", "we", "well", "were", "weren", 
	"weren't", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", 
	"whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", 
	"whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", 
	"without", "won", "won't", "would", "wouldn", "wouldn't", "y", "yet", "you", "your", 
	"yours", "yourself", "yourselves", "from SVM import SVCone", "two", "three", "four", "five", "six", "seven",
	"eight", "nine", "zero", "between", 'below', 'ourselves', "you'll", 'again', 'once', 'over', 'shan', 'few', 
    'against', 'before', 'out', 'down', 'both', 'up', "you've", "shan't", "you're", "should've", 'ours', 'ma', 
    "couldn't", 'during', 'more', 'ain', 'through', 'after', 'above', "she's", "you'd", 'under' ]

In [4]:
def get_citations(folder):
    citation_list = {}
    for file in os.listdir(folder) :
        tree = ET.parse(folder+file)
        root = tree.getroot()
        id = file[:8]
        for element in root.iterfind("algorithm"):
            if(element.attrib['name']=="ParsCit"):
                citlist = element.getchildren()
                cits = citlist[0].getchildren()
                citations = []
                for cit in cits:
                    cit_dict = {}
                    if(cit.attrib['valid']=="true"):
                        try :
                            title = cit.find('title').text.lower()
                        except :
                            title = cit.find('rawString').text.lower()
                        cit_dict['title'] = title
                        cit_dict['cit'] = cit
                        citations.append(cit_dict)
                
                citation_list[id] = citations
            
    return citation_list 

In [5]:
citations = get_citations("../xmls/")

In [6]:
def get_words(string, cit_auths=None):

    string = string.replace('-', '').lower()
    context_words = word_tokenize(string)
    final_context_words = []

    for word in context_words:
        if word in stop_words:
            continue
        if re.fullmatch(year_regex, word): #or match_auths(word, cit_auths):
            continue
        if re.fullmatch(r'[0-9]+([.][0-9]+)?', word):
            word = '<number>'
            final_context_words.append(word)
        elif 'this_citation' in word:
            final_context_words.append('<this_citation>')
        elif re.fullmatch(r'[a-z]+', word):
            original_word = word
            final_word = lemmatizer.lemmatize(word)
            
            if final_word not in conversion_dict:
                conversion_dict[final_word] = {}

            if original_word not in conversion_dict[final_word]:
                conversion_dict[final_word][original_word] = 0
            conversion_dict[final_word][original_word]+=1

            final_context_words.append(final_word)

    return final_context_words

In [7]:
def get_contexts(citations) :
    dataset = {}
    count = 0
    for key in citations.keys():
        context_list = []
        for cit in citations[key] :
            dict1 = {}
            dict1['paper_name'] = cit['title']
            dict1['context'] = ""
            for context in cit['cit'].findall('contexts/context') :
                text = context.text.lower()
                citstr = context.get('citStr').lower()
                text = text.replace(citstr, "this_citation")
                dict1['context']+= text+" "
            dict1['context'] = get_words(dict1['context'])
            context_list.append(dict1)
        dataset[key] = context_list
        count+=1
        if(count%100==0) :
            print(count)
    return dataset

In [8]:
contexts = get_contexts(citations)

100
200
300
400
500
600
700
800
900
1000
1100


In [9]:
max1 = 0
avg = 0
number = 0
for key in contexts :
    papers = contexts[key]
    for pap in papers :
        val = len(pap['context'])
        avg += val
        number += 1
        if(val>max1) :
            max1 = val
            
print(max1)
print(avg/number)

1141
140.41749366058292


In [10]:
embeddings_dict = {}
with open("glove.6B.100d.txt", 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

In [11]:
print(len(embeddings_dict.keys()))

400000


In [12]:
mapping = {}
mapping_rev = {}
values = []
count = 0
for key in contexts :
    papers = contexts[key]
    for pap in papers :
        name = pap['paper_name']
        words = pap['context']
        if(len(words)>0) :
            mapping[key+"_|_"+name] = count
            mapping_rev[count] = key+"_|_"+name
            count+=1
            embed = []
            for word in words :
                if(word in embeddings_dict) :
                    embed.append(embeddings_dict[word])
                else :
                    embed.append(embeddings_dict['unk'])
            diff = 1141-len(words)
            if(diff>0) :
                for i in range(diff) :
                    embed.append(embeddings_dict['unk'])
            values.append(embed)  

In [13]:
print(len(values))

27635


In [14]:
tags = pickle.load(open("../pickles_data/baseline_tags.pkl","rb"))

In [15]:
output = []
for i in range(count) :
    key,name = mapping_rev[i].split("_|_")
    papers = tags[key]
    for pap in papers :
        if(pap['paper_name']==name) :
            output.append(pap['tag'])
            break

In [16]:
print(len(output))

27635


In [17]:
size = len(values)
trainset = values[:int(0.8*size)]
trainlabel = output[:int(0.8*size)]
testset = values[int(0.8*size):]
testlabel = output[int(0.8*size):]

In [18]:
class Attention(nn.Module):
    def __init__(self, feature_dim, step_dim, bias=True, **kwargs):
        super(Attention, self).__init__(**kwargs)
        
        self.supports_masking = True

        self.bias = bias
        self.feature_dim = feature_dim
        self.step_dim = step_dim
        self.features_dim = 0
        
        weight = torch.zeros(feature_dim, 1)
        nn.init.kaiming_uniform_(weight)
        self.weight = nn.Parameter(weight)
        
        if bias:
            self.b = nn.Parameter(torch.zeros(step_dim))
        
    def forward(self, x, mask=None):
        feature_dim = self.feature_dim 
        step_dim = self.step_dim

        eij = torch.mm(
            x.contiguous().view(-1, feature_dim), 
            self.weight
        ).view(-1, step_dim)
        
        if self.bias:
            eij = eij + self.b

#         print("eij",eij.shape)
#         print("x",x.contiguous().view(-1,feature_dim).shape)
            
        eij = torch.tanh(eij)
        a = torch.exp(eij)
        
        if mask is not None:
            a = a * mask

        a = a / (torch.sum(a, 1, keepdim=True) + 1e-10)

        weighted_input = x * torch.unsqueeze(a, -1)
        return torch.sum(weighted_input, 1)

class Attention_Net(nn.Module):
    def __init__(self):
        super(Attention_Net, self).__init__()
        drp = 0.1
        max_features = 1141
        embed_size = 100
        # self.embedding = nn.Embedding(max_features, embed_size)
        # # self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        # self.embedding.weight.requires_grad = False

        # self.embedding_dropout = nn.Dropout2d(0.1)
        self.lstm = nn.LSTM(embed_size, 32, bidirectional=True, batch_first=True)
#         self.lstm2 = nn.LSTM(64*2, 32, bidirectional=True, batch_first=True)

        self.attention_layer = Attention(64, 1141)
        
        self.linear = nn.Linear(64 , 32)
        self.relu = nn.ReLU()
        self.out = nn.Linear(32, 2)

    def forward(self, x):
        h_embedding = x
        h_lstm, _ = self.lstm(h_embedding)
#         h_lstm, _ = self.lstm2(h_lstm)
        h_lstm_atten = self.attention_layer(h_lstm)
        conc = self.relu(self.linear(h_lstm_atten))
        out = self.out(conc)
        return out

In [19]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
batch_size = 128

train_data = TensorDataset(torch.as_tensor(trainset), torch.as_tensor(trainlabel))
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(torch.as_tensor(testset), torch.as_tensor(testlabel))
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=128)

In [20]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

def eval_model(model, val_loader) :
    final_out = []
    final_lab = []

    for idx, (val_input, val_label) in enumerate(val_loader):
    # val_input = val_input.permute(1,0,2)
    # print(train_input.shape)
        val_input = val_input.type(torch.FloatTensor)
        val_input = val_input.cuda()
        output = model(val_input)
        output = output.cpu().detach().numpy()
        val_label = val_label.cpu().detach().numpy()
        ar= []
        for i in range(output.shape[0]) :
            index = -1
            max_val = -1
            for j in range(len(output[i])) :
                if(output[i][j]>max_val) :
                    index = j
                    max_val = output[i][j]
            ar.append(index)
    
        val_label = list(val_label)
        final_out.extend(ar)
        final_lab.extend(val_label)

        
    return classification_report(final_lab, final_out)

In [21]:
learning_rate = 0.005
num_epochs = 10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
model = Attention_Net().to(device)
weights = [1.0, 5.0]
class_weights = torch.FloatTensor(weights).cuda()
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for batch_id, (train_input, train_label) in enumerate(train_dataloader):
        optimizer.zero_grad() 
        train_input = train_input.type(torch.FloatTensor)
        train_input = train_input.cuda()
        output = model(train_input)

        train_label = train_label.type(torch.LongTensor)
        train_label = train_label.cuda()
        loss = criterion(output, train_label)
        loss.backward()
        optimizer.step()
        
        if batch_id % 100 == 0:
            print('Epoch [{}/{}], Loss:{:.4f}'
                .format(epoch+1, num_epochs, loss.data))
            print("classification_report")
            print(eval_model(model,val_dataloader))
            print("--------------------------------------------------------------")
            # losses.append(loss.data)
            
        del train_input
        del train_label
        del output
        del loss
        torch.cuda.empty_cache()

Epoch [1/10], Loss:0.6837
classification_report


/home/tanisha17116/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.90      1.00      0.95      5000
           1       0.00      0.00      0.00       527

    accuracy                           0.90      5527
   macro avg       0.45      0.50      0.47      5527
weighted avg       0.82      0.90      0.86      5527

--------------------------------------------------------------
Epoch [1/10], Loss:0.4220
classification_report
              precision    recall  f1-score   support

           0       0.97      0.84      0.90      5000
           1       0.33      0.77      0.47       527

    accuracy                           0.83      5527
   macro avg       0.65      0.80      0.68      5527
weighted avg       0.91      0.83      0.86      5527

--------------------------------------------------------------
Epoch [2/10], Loss:0.6311
classification_report
              precision    recall  f1-score   support

           0       0.96      0.92      0.94      5000
           1   

Epoch [10/10], Loss:0.1500
classification_report
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      5000
           1       0.58      0.57      0.58       527

    accuracy                           0.92      5527
   macro avg       0.77      0.76      0.77      5527
weighted avg       0.92      0.92      0.92      5527

--------------------------------------------------------------


In [23]:
learning_rate = 0.005
num_epochs = 10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
model = Attention_Net().to(device)
weights = [1.0, 2.0]
class_weights = torch.FloatTensor(weights).cuda()
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for batch_id, (train_input, train_label) in enumerate(train_dataloader):
        optimizer.zero_grad() 
        train_input = train_input.type(torch.FloatTensor)
        train_input = train_input.cuda()
        output = model(train_input)

        train_label = train_label.type(torch.LongTensor)
        train_label = train_label.cuda()
        loss = criterion(output, train_label)
        loss.backward()
        optimizer.step()
        
        if batch_id % 100 == 0:
            print('Epoch [{}/{}], Loss:{:.4f}'
                .format(epoch+1, num_epochs, loss.data))
            print("classification_report")
            print(eval_model(model,val_dataloader))
            print("--------------------------------------------------------------")
            # losses.append(loss.data)
            
        del train_input
        del train_label
        del output
        del loss
        torch.cuda.empty_cache()

Epoch [1/10], Loss:0.8352
classification_report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      5000
           1       0.10      1.00      0.17       527

    accuracy                           0.10      5527
   macro avg       0.05      0.50      0.09      5527
weighted avg       0.01      0.10      0.02      5527

--------------------------------------------------------------
Epoch [1/10], Loss:0.3317
classification_report
              precision    recall  f1-score   support

           0       0.94      0.96      0.95      5000
           1       0.49      0.37      0.43       527

    accuracy                           0.90      5527
   macro avg       0.71      0.67      0.69      5527
weighted avg       0.89      0.90      0.90      5527

--------------------------------------------------------------
Epoch [2/10], Loss:0.3047
classification_report
              precision    recall  f1-score   support

           0       0.

Epoch [10/10], Loss:0.2032
classification_report
              precision    recall  f1-score   support

           0       0.95      0.95      0.95      5000
           1       0.54      0.56      0.55       527

    accuracy                           0.91      5527
   macro avg       0.75      0.75      0.75      5527
weighted avg       0.91      0.91      0.91      5527

--------------------------------------------------------------


In [22]:
print(eval_model(model, val_dataloader))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95      5000
           1       0.52      0.62      0.57       527

    accuracy                           0.91      5527
   macro avg       0.74      0.78      0.76      5527
weighted avg       0.92      0.91      0.91      5527



In [ ]:
torch.save(model.state_dict, "models/2layer_bilstm_atten_statedict.pt")

In [ ]:
torch.save(model, "models/2layer_bilstm_atten_model.pt")

In [ ]:
class Attention_Net_Features(nn.Module) :
    
    